## **CV HW4: Multi-object Tracking (MOT) with Detection**
**Detection**: YOLOv5, 
**Tracking**: Simple Online Realtime Tracking (SORT)

---



## **1. Unzip data folder**

In [1]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [2]:
# Change the path according to your setup 
!unzip '/drive/MyDrive/hw4/sort-master.zip'
!unzip '/drive/MyDrive/hw4/KITTI_17_images.zip'

Archive:  /drive/MyDrive/hw4/sort-master.zip
 extracting: sort-master/.gitignore  
   creating: sort-master/data/
   creating: sort-master/data/train/
   creating: sort-master/data/train/ADL-Rundle-6/
   creating: sort-master/data/train/ADL-Rundle-6/det/
  inflating: sort-master/data/train/ADL-Rundle-6/det/det.txt  
   creating: sort-master/data/train/ADL-Rundle-8/
   creating: sort-master/data/train/ADL-Rundle-8/det/
  inflating: sort-master/data/train/ADL-Rundle-8/det/det.txt  
   creating: sort-master/data/train/ETH-Bahnhof/
   creating: sort-master/data/train/ETH-Bahnhof/det/
  inflating: sort-master/data/train/ETH-Bahnhof/det/det.txt  
   creating: sort-master/data/train/ETH-Pedcross2/
   creating: sort-master/data/train/ETH-Pedcross2/det/
  inflating: sort-master/data/train/ETH-Pedcross2/det/det.txt  
   creating: sort-master/data/train/ETH-Sunnyday/
   creating: sort-master/data/train/ETH-Sunnyday/det/
  inflating: sort-master/data/train/ETH-Sunnyday/det/det.txt  
   creating: s

# **2. Install requirements**

In [3]:
# !pip install -r sort-master/requirements.txt
# !pip install cv

In [4]:
%cd pytorch_objectdetecttrack
!pip install filterpy==1.1.0

[Errno 2] No such file or directory: 'pytorch_objectdetecttrack'
/content
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.1.0-py3-none-any.whl size=85439 sha256=9c6408c4d450c3bad21e1ad6d7261f55fc1328ca83ee62257d0577f36d52667b
  Stored in directory: /root/.cache/pip/wheels/4d/26/1e/afde04eff5384bdd90a0c517b6fbe923fc2203c3200992509e
Successfully built filterpy


# **3. Import libraries**

In [5]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import sys
sys.path.insert(0,'./sort-master/')
import matplotlib
from google.colab.patches import cv2_imshow
from collections import namedtuple, OrderedDict

# **4. Load YOLOv5 detector from torch hub**

In [6]:
yolov5_detector = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained = True)
yolov5_detector.float()
yolov5_detector.eval()

/usr/local/lib/python3.10/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-5-6 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)



requirements: /root/.cache/torch/hub/requirements.txt not found, check failed.


100%|██████████| 14.1M/14.1M [00:00<00:00, 282MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

Set configuration like confidence threshold



In [7]:


# Set the configuration for the detector
yolov5_detector.conf = 0.25 # confidence threshold
yolov5_detector.agnostic_nms = True

# **5. Import SORT library**

In [9]:
sort_dir = os.path.join(os.getcwd(), 'sort-master/sort.py')
sys.path.append(sort_dir)

# Now you can import the sort module
from sort import *

#**6. Perform tracking with detection**

Import dependencies

In [10]:

import os, sys, time, datetime, random
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch import Tensor
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image


Convert image file to video file 

In [11]:
image_folder = '/content/KITTI_17_images'
video_name = 'video.mp4'

# Read the first image to get its size
img = cv2.imread(os.path.join(image_folder, os.listdir(image_folder)[0]))
height, width, channels = img.shape

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter(video_name, fourcc, 25.0, (width, height))

# Loop over all images in the folder and write them to the video
for filename in sorted(os.listdir(image_folder)):
    if filename.endswith('.jpg'):
        img_path = os.path.join(image_folder, filename)
        img = cv2.imread(img_path)
        video.write(img)

# Release the video object and close all windows
video.release()
cv2.destroyAllWindows()

In [12]:
# Write your code here to perform tracking with detection using the provided YOLOv5 model and the SORT implementation



import glob
import random

# Define the number of colors you want
num_colors = 15

# Create a list of random RGB tuples
colours = [(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for i in range(num_colors)]
mot_tracker=Sort()

cap = cv2.VideoCapture('/content/video.mp4')
ret, frame = cap.read()
frame_height, frame_width, _ = frame.shape
out = cv2.VideoWriter('result.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
print("Processing Video...")
frame_counter=0  

with open('arjun-01.txt', 'w') as f:
  #lists to store data to be entered in the the output file in gt.txt format
  bb_frame=[]
  bb_left=[]
  bb_top=[]
  bb_idx=[]
  bb_width=[]
  bb_height=[]
  bb_score=[]

  #loop for every read frame
  while cap.isOpened():
    frame_counter+=1
    ret, frame = cap.read()
    if not ret:
      out.release()
      break
    preds=yolov5_detector(frame)
    detections=preds.pred[0].cpu().numpy()
    #select detections with label '0'
    person_detections = detections[detections[:, 5] == 0] 
    track_bbs_ids=mot_tracker.update(person_detections)
    num_colours=len(colours)
    
    #loop for every pedestrian in a frame
    for j in range(len(track_bbs_ids.tolist())):
      coords=track_bbs_ids.tolist()[j]
      # if(int(labels[j])== 0):
      x1, y1, x2, y2=int(coords[0]), int(coords[1]), int(coords[2]), int(coords[3])
      name_idx=int(coords[4])
      bb_frame.append(frame_counter)
      bb_top.append(y1)
      bb_left.append(x1)
      bb_idx.append(name_idx)
      bb_height.append(np.abs(y2-y1))
      bb_width.append(np.abs(x2-x1))
      for var in person_detections:
        if var[4]>=0.5:
          bb_score.append(1)
        else:   
          bb_score.append(0)
     
      
      # idx.append(name_idx)
      name=f'ID:{str(name_idx)}'
      name_idx = name_idx % num_colours
      color = colours[name_idx]
      
      cv2.rectangle(frame,(x1,y1),(x2,y2),color,2)
      cv2.putText(frame, name, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
      
      
      out.write(frame)
      cv2.waitKey(0)
    
    
    
      

  out.release()

  print("Done processing video")
  for i in range(len(bb_width)):
        # Write the values from each list separated by a tab character
        f.write(f"{bb_frame[i]}, {bb_idx[i]}, {bb_left[i]},{bb_top[i]},{bb_width[i]},{bb_height[i]},{bb_score[i]},-1,-1,-1\n")


Processing Video...
Done processing video


# **7. Report Evaluation Metrics**

In [13]:
# Use the Track-Eval kit to report the complete set of performance and accuracy metrics
# Comment on and interpret MOTA and MOTP values

!unzip '/drive/MyDrive/hw4/TrackEval-master.zip'
!unzip /drive/MyDrive/data.zip -d /content/TrackEval-master/


Archive:  /drive/MyDrive/hw4/TrackEval-master.zip
12c8791b303e0a0b50f753af204249e622d0281a
   creating: TrackEval-master/
  inflating: TrackEval-master/.gitignore  
  inflating: TrackEval-master/LICENSE  
  inflating: TrackEval-master/Readme.md  
   creating: TrackEval-master/docs/
  inflating: TrackEval-master/docs/BDD100k-format.txt  
  inflating: TrackEval-master/docs/DAVIS-format.txt  
   creating: TrackEval-master/docs/How_To/
  inflating: TrackEval-master/docs/How_To/Add_a_new_metric.md  
  inflating: TrackEval-master/docs/KITTI-format.txt  
   creating: TrackEval-master/docs/MOTChallenge-Official/
  inflating: TrackEval-master/docs/MOTChallenge-Official/Readme.md  
  inflating: TrackEval-master/docs/MOTChallenge-format.txt  
  inflating: TrackEval-master/docs/MOTS-format.txt  
   creating: TrackEval-master/docs/OpenWorldTracking-Official/
  inflating: TrackEval-master/docs/OpenWorldTracking-Official/Readme.md  
   creating: TrackEval-master/docs/RobMOTS-Official/
  inflating: Tr

In [14]:

#folders for ground truth 
# set the path to the existing folder
path = '/content/TrackEval-master/data/gt/mot_challenge/'

# create the parent folder if it doesn't already exist
parent_folder_name = 'arjun-train'
parent_folder_path = os.path.join(path, parent_folder_name)
if not os.path.exists(parent_folder_path):
    os.mkdir(parent_folder_path)

# create a new folder within the parent folder
new_folder_name = 'arjun-01'
new_folder_path = os.path.join(parent_folder_path, new_folder_name)
os.mkdir(new_folder_path)


# create a new text file within the new folder and write some text to it
new_file_name = 'seqinfo.ini'
new_file_path = os.path.join(new_folder_path, new_file_name)
with open(new_file_path, 'w') as f:
    f.write('[Sequence]\nname=arjun\nseqLength=145\n')


In [15]:

#store ground truth 
novel_folder_name = 'gt'
novel_folder_path = os.path.join(new_folder_path, novel_folder_name)
os.mkdir(novel_folder_path)

!cp '/drive/MyDrive/hw4/gt.txt' '/content/TrackEval-master/data/gt/mot_challenge/arjun-train/arjun-01/gt/'

#create seqmaps for testing 
# Set the directory path
directory = '/content/TrackEval-master/data/gt/mot_challenge/seqmaps/'

file_path = os.path.join(directory, 'arjun-train.txt')
with open(file_path, 'w') as f:
    f.write('name\n')
    f.write('arjun-01\n')
      

file_path = os.path.join(directory, 'arjun-test.txt')
with open(file_path, 'w') as f:
    f.write('name\n')
    f.write('arjun-01\n')
    

file_path = os.path.join(directory, 'arjun-all.txt')
with open(file_path, 'w') as f:
    f.write('name\n')
    f.write('arjun-01\n')
    

In [16]:
#folders for trackers

# Set the directory path
directory = '/content/TrackEval-master/data/trackers/mot_challenge/'

file_path = os.path.join(directory, 'arjun-train')
os.mkdir(file_path)
new_file_path=os.path.join(file_path, 'MyTracker')
os.mkdir(new_file_path)

novel_file_path=os.path.join(new_file_path, 'data')
os.mkdir(novel_file_path)
!cp '/content/arjun-01.txt' '/content/TrackEval-master/data/trackers/mot_challenge/arjun-train/MyTracker/data/'



In [18]:
!python /content/TrackEval-master/scripts/run_mot_challenge.py --DO_PREPROC False --BENCHMARK arjun --SPLIT_TO_EVAL train --TRACKERS_TO_EVAL MyTracker --METRICS HOTA CLEAR Identity VACE --USE_PARALLEL False --NUM_PARALLEL_CORES 1 


Eval Config:
USE_PARALLEL         : False                         
NUM_PARALLEL_CORES   : 1                             
BREAK_ON_ERROR       : True                          
RETURN_ON_ERROR      : False                         
LOG_ON_ERROR         : /content/TrackEval-master/error_log.txt
PRINT_RESULTS        : True                          
PRINT_ONLY_COMBINED  : False                         
PRINT_CONFIG         : True                          
TIME_PROGRESS        : True                          
DISPLAY_LESS_PROGRESS : False                         
OUTPUT_SUMMARY       : True                          
OUTPUT_EMPTY_CLASSES : True                          
OUTPUT_DETAILED      : True                          
PLOT_CURVES          : True                          

MotChallenge2DBox Config:
PRINT_CONFIG         : True                          
GT_FOLDER            : /content/TrackEval-master/data/gt/mot_challenge/
TRACKERS_FOLDER      : /content/TrackEval-master/data/trackers/mot_